**Christian Data Exploration**

This notebook outlines the actions taken to transform the data so that it is ready for model use. Specifically:
- It will convert binary columns like win_loss, OT, Homegame, etc. into 1's and 0's where 1 indicates the symantical presense of "something." 
- Convert numeric string values from NaN to 0.
- Convert conference into a series of binary variables.
- Attempt to identify a solution to the AP top 25 flow-in and out perdicament.
- Shift the dataframe to identify the rank changes (already completed by Serena in the data).      

**Steps and Workflow**
1. Data setup
2. Data Exploration
    - Explore AP Rank Data
    - Get all Unique Conference Values
3. Data Manipulation
    - Setup Elo Merge
    - Setting up AP Rank Shift ~~(not needed since it is already done in the data)~~
    - Remove year 2020 which has oddities due to COVID-19
    - Setup all Binary Variables
        - Conference variables (for both the current and opposing teams)
        - In same conference variable or outside conference variable
    - Setup 3 week rolling averages / win loss pattern, etc.
    - Merge Elo Data into the dataset
    - 
4. Data Validation Check

**Data Concepts**
- The dataset contains two records for each game: (1) the winner and (2) the looser. 
- AP Ranks only display for the top 25 teams and the AP rank either represents before or after. Eitherway a shift is needed to determine the before and after on each game and create the differential. 

**Outstanding Questons / Data Issues**
- What does "Bye" mean in the opponent field? It seems like perhaps they did not have a game at during this week and perhaps could be dropped? 
- There still seems to be negative values in the week columns -- may want to just drop 2020 from the dataset since it may cause errors. 
- I may need to drop the first week of the season or remove it's shift values since it's pulling in the previous season's information.
- I would need the opposition conference aswell in the columns in order to do the comparison between conferences. 


**1. Data Setup**

In [44]:
import pandas as pd
import seaborn as sn

# Set Standard dataframe settings
pd.set_option('display.max_columns', None)
df_clean = pd.read_csv("https://raw.githubusercontent.com/sjhawkins24/Group-8-Project/refs/heads/main/03%20-%20Cleaned%20Data%20Space/mergedTrainingData.csv")

#df_clean.drop(columns =['Unnamed: 0','code'], inplace=True)
df_clean.sort_values(by=['season','Team','week']).reset_index().head(20)

,index,Unnamed: 0,week,season,Team,opponent,code,date,win_loss,OT,OT_num,pass,rush,rec,points_allowed,points_scored,point_differential,home_game,CONF,SOR,FPI,SOS,GC,AVGWP,AP_rank,opponent_rank,rank_change
0,2238,2239,2,2021,Air Force,Navy,2005,"Sat, Sep 11",W,False,NaN,49.0,97.0,27.0,3,23,20,False,Mountain West,33,66,110,24,6,NaN,NaN,NaN
1,2600,2601,3,2021,Air Force,Utah State,2005,"Sat, Sep 18",L,False,NaN,182.0,102.0,NaN,49,45,-4,True,Mountain West,33,66,110,24,6,NaN,NaN,0.0
2,3007,3008,4,2021,Air Force,Florida Atlantic,2005,"Sat, Sep 25",W,False,NaN,70.0,164.0,46.0,7,31,24,True,Mountain West,33,66,110,24,6,NaN,NaN,0.0
3,3454,3455,5,2021,Air Force,New Mexico,2005,"Sat, Oct 2",W,False,NaN,33.0,142.0,33.0,10,38,28,False,Mountain West,33,66,110,24,6,NaN,NaN,0.0
4,3912,3913,6,2021,Air Force,Wyoming,2005,"Sat, Oct 9",W,False,NaN,110.0,140.0,77.0,14,24,10,True,Mountain West,33,66,110,24,6,NaN,NaN,0.0
5,4340,4341,7,2021,Air Force,Boise State,2005,"Sat, Oct 16",W,False,NaN,59.0,138.0,59.0,17,24,7,False,Mountain West,33,66,110,24,6,NaN,NaN,0.0
6,4763,4764,8,2021,Air Force,San Diego State,2005,"Sat, Oct 23",L,False,NaN,58.0,50.0,31.0,20,14,-6,True,Mountain West,33,66,110,24,6,NaN,21.0,0.0
7,0,1,10,2021,Air Force,Army,2005,"Sat, Nov 6",L,True,,226.0,68.0,106.0,21,14,-7,True,Mountain West,33,66,110,24,6,NaN,NaN,0.0
8,464,465,11,2021,Air Force,Colorado State,2005,"Sat, Nov 13",W,False,NaN,121.0,151.0,92.0,21,35,14,False,Mountain West,33,66,110,24,6,NaN,NaN,0.0
9,929,930,12,2021,Air Force,Nevada,2005,"Fri, Nov 19",W,True,3,23.0,208.0,23.0,39,41,2,False,Mountain West,33,66,110,24,6,NaN,NaN,0.0


**2. Data Exploration - Explore AP Rank Data** 

In [45]:
# Select all rows where AP_rank is not null and then display the first 14 rows sorted by team and week
mask = df_clean['AP_rank'].notna()
df_ap_rank = df_clean[mask]
df_ap_rank.iloc[0:14].sort_values(by=['Team','week']).reset_index()

,index,Unnamed: 0,week,season,Team,opponent,code,date,win_loss,OT,OT_num,pass,rush,rec,points_allowed,points_scored,point_differential,home_game,CONF,SOR,FPI,SOS,GC,AVGWP,AP_rank,opponent_rank,rank_change
0,2,3,10,2021,Alabama,LSU,333,"Sat, Nov 6",W,False,NaN,302.0,18.0,160.0,14,20,6,True,SEC,2,2,1,2,3,3.0,NaN,-1.0
1,7,8,10,2021,Auburn,Texas A&M,2,"Sat, Nov 6",L,False,NaN,153.0,69.0,50.0,20,3,-17,False,SEC,48,20,4,29,58,12.0,13.0,-9.0
2,9,10,10,2021,Baylor,TCU,239,"Sat, Nov 6",L,False,NaN,214.0,125.0,121.0,30,28,-2,False,Big 12,7,15,25,6,7,14.0,NaN,-4.0
3,16,17,10,2021,Cincinnati,Tulsa,2132,"Sat, Nov 6",W,False,NaN,274.0,43.0,113.0,20,28,8,True,American,6,10,54,5,2,2.0,NaN,0.0
4,17,18,10,2021,Coastal Carolina,Georgia State,324,"Sat, Nov 13",L,False,NaN,233.0,128.0,101.0,42,40,-2,True,Sun Belt,34,45,130,47,5,21.0,NaN,-3.0
5,27,28,10,2021,Fresno State,San Diego State,278,"Sat, Oct 30",W,False,NaN,306.0,186.0,107.0,20,30,10,False,Mountain West,30,53,95,37,19,25.0,NaN,-1.0
6,28,29,10,2021,Georgia,Missouri,61,"Sat, Nov 6",W,False,NaN,255.0,41.0,76.0,6,43,37,True,SEC,1,1,3,1,1,1.0,NaN,0.0
7,33,34,10,2021,Houston,South Florida,248,"Sat, Nov 6",W,False,NaN,385.0,130.0,164.0,42,54,12,False,American,17,37,78,27,10,20.0,NaN,-6.0
8,36,37,10,2021,Iowa,Northwestern,2294,"Sat, Nov 6",W,False,NaN,172.0,141.0,68.0,12,17,5,False,Big Ten,16,32,23,33,54,19.0,NaN,9.0
9,41,42,10,2021,Kentucky,Tennessee,96,"Sat, Nov 6",L,False,NaN,372.0,109.0,166.0,45,42,-3,True,SEC,19,28,32,15,15,18.0,NaN,6.0


**2. Data Exploration - Get all Unique Conference Values**

These values will be used to create a set of binary columns for each conference for both home and away teams.

In [46]:
## Old

# Groups by conference and counts unique teams in each conference.
df_clean.groupby('CONF')['Team'].nunique().reset_index(name='unique_teams')

# Create the conference list for binary column creation
conference_list = []

for conf in df_clean['CONF']:
    if conf not in conference_list:
        conference_list.append(conf)

print(conference_list)

['Mountain West', 'MAC', 'SEC', 'Pac-12', 'FBS Indep.', 'Big 12', 'ACC', 'CUSA', 'American', 'Sun Belt', 'Big Ten']


In [47]:
# Create a Table Storing all Conference Information

df_conference = df_clean[['Team',"CONF"]].rename(columns={'Team' : 'Team ID','CONF' : 'Conference ID'}).drop_duplicates().sort_values('Conference ID')
df_conference

,Team ID,Conference ID
62,North Carolina,ACC
31,Georgia Tech,ACC
225,Virginia,ACC
373,California,ACC
11,Boston College,ACC
...,...,...
188,Old Dominion,Sun Belt
94,Troy,Sun Belt
154,James Madison,Sun Belt
100,UL Monroe,Sun Belt


**3. Import Elo Data into Base Dataframe**

In [48]:
# Retrieve Elo Data

elo_df = pd.read_csv('https://raw.githubusercontent.com/sjhawkins24/Group-8-Project/9924dcab26dafc03bc17c78bb44d5fb5ec1c99e8/elo_ratings_weekly.csv')

# Merge Elo Data onto Base Dataframe
df_clean = pd.merge(
    df_clean,
    elo_df,
    on=['season','week','Team'],
    how='inner'
)

# Check for any error after the merge
missing = df_clean['Elo_pregame'].isna().sum()

if missing > 0:
    raise ValueError(f"ERROR: Missing Elo values found: {missing}")
else: 
    print("Merge completed successfully.")

Merge completed successfully.


In [49]:
df_clean.head()

,Unnamed: 0,week,season,Team,opponent,code,date,win_loss,OT,OT_num,pass,rush,rec,points_allowed,points_scored,point_differential,home_game,CONF,SOR,FPI,SOS,GC,AVGWP,AP_rank,opponent_rank,rank_change,Elo_pregame
0,1,10,2021,Air Force,Army,2005,"Sat, Nov 6",L,True,,226.0,68.0,106.0,21,14,-7,True,Mountain West,33,66,110,24,6,NaN,NaN,0.0,1600.530230
1,2,10,2021,Akron,Western Michigan,2006,"Tue, Nov 9",L,False,NaN,305.0,58.0,122.0,45,40,-5,False,MAC,127,128,67,118,125,NaN,NaN,0.0,1354.196906
2,3,10,2021,Alabama,LSU,333,"Sat, Nov 6",W,False,NaN,302.0,18.0,160.0,14,20,6,True,SEC,2,2,1,2,3,3.0,NaN,-1.0,1618.269758
3,4,10,2021,Arizona,California,12,"Sat, Nov 6",W,False,NaN,129.0,68.0,46.0,3,10,7,True,Pac-12,122,106,71,110,122,NaN,NaN,0.0,1363.708830
4,5,10,2021,Arizona State,USC,9,"Sat, Nov 6",W,False,NaN,145.0,202.0,68.0,16,31,15,True,Pac-12,49,39,61,61,63,NaN,NaN,0.0,1550.632633


**3. Setup AP Rank based on Elo, Shift, and Differential**

In [68]:
# Define main dataframe and initiate custom AP rank field
df_base = df_clean.sort_values(by=['Team','season','week']).reset_index()
df_base['Cus_AP_Rank'] = df_base['AP_rank']

# Function that operates on each week-rank group
def assign_unranked_by_week(group):
    grp = group.copy()

    # Mask for all Unranked Teams
    unranked_mask = grp['AP_rank'].isna() | (grp['AP_rank'] > 25)

    # 
    grp.loc[unranked_mask, 'elo_rank_unranked'] = (
        grp.loc[unranked_mask, 'Elo_pregame']
            .rank(method='first', ascending=False)
            .astype(int)
    )

    # Adjust rank for each bucket
    grp.loc[unranked_mask, 'Cus_AP_Rank'] = grp.loc[unranked_mask, 'elo_rank_unranked'] + 25

    return grp

# Apply ranking per week within each grouping
df_base = df_base.groupby(['season','week'], group_keys=False).apply(assign_unranked_by_week)


df_sort = df_base.sort_values(by=['season','week','Cus_AP_Rank']).reset_index()


mask = ((df_sort['week'] != 0) & (df_sort['Team'] == 'Notre Dame'))
df_sort.loc[mask].iloc[:15]



#df_base["Previous_AP_Rank"] = df_base.groupby('Team')['AP_rank'].shift(1)
#df_base['AP_Rank_Differential'] = df_base['AP_rank'] - df_base['Previous_AP_Rank']

#mask = df_base['Team'] == 'Alabama'

#df_base[mask].head(13)

# Check on the Alabama Crimson Tide for the first 14 weeks to determien if the previous AP rank shift is working correctly


C:\Users\chris\AppData\Local\Temp\ipykernel_24204\1353955192.py:25: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_base = df_base.groupby(['season','week'], group_keys=False).apply(assign_unranked_by_week)


,level_0,index,Unnamed: 0,week,season,Team,opponent,code,date,win_loss,OT,OT_num,pass,rush,rec,points_allowed,points_scored,point_differential,home_game,CONF,SOR,FPI,SOS,GC,AVGWP,AP_rank,opponent_rank,rank_change,Elo_pregame,Cus_AP_Rank,elo_rank_unranked
3,3193,2285,2286,2,2021,Notre Dame,Toledo,87,"Sat, Sep 11",W,False,NaN,239.0,78.0,81.0,29,32,3,True,FBS Indep.,10,6,43,9,11,7.0,NaN,0.0,1500.000000,7.0,NaN
93,3194,2661,2662,3,2021,Notre Dame,Purdue,87,"Sat, Sep 18",W,False,NaN,223.0,91.0,120.0,13,27,14,True,FBS Indep.,10,6,43,9,11,10.0,NaN,3.0,1512.541665,10.0,NaN
195,3195,3072,3073,4,2021,Notre Dame,Wisconsin,87,"Sat, Sep 25",W,False,NaN,158.0,33.0,76.0,13,41,28,True,FBS Indep.,10,6,43,9,11,10.0,15.0,0.0,1540.464801,10.0,NaN
303,3196,3523,3524,5,2021,Notre Dame,Cincinnati,87,"Sat, Oct 2",L,False,NaN,143.0,45.0,93.0,24,13,-11,True,FBS Indep.,10,6,43,9,11,7.0,8.0,-3.0,1576.237514,7.0,NaN
427,3197,3969,3970,6,2021,Notre Dame,Virginia Tech,87,"Sat, Oct 9",W,False,NaN,113.0,81.0,70.0,29,32,3,False,FBS Indep.,10,6,43,9,11,13.0,NaN,6.0,1542.899076,13.0,NaN
633,3198,4819,4820,8,2021,Notre Dame,USC,87,"Sat, Oct 23",W,False,NaN,189.0,138.0,57.0,16,31,15,True,FBS Indep.,10,6,43,9,11,13.0,NaN,0.0,1558.900201,13.0,NaN
738,3199,5268,5269,9,2021,Notre Dame,North Carolina,87,"Sat, Oct 30",W,False,NaN,213.0,199.0,74.0,34,44,10,True,FBS Indep.,10,6,43,9,11,11.0,NaN,-2.0,1578.705239,11.0,NaN
848,3200,66,67,10,2021,Notre Dame,Navy,87,"Sat, Nov 6",W,False,NaN,269.0,95.0,139.0,6,34,28,True,FBS Indep.,10,6,43,9,11,8.0,NaN,-3.0,1596.528021,8.0,NaN
963,3201,531,532,11,2021,Notre Dame,Virginia,87,"Sat, Nov 13",W,False,NaN,132.0,70.0,84.0,3,28,25,False,FBS Indep.,10,6,43,9,11,7.0,NaN,-1.0,1612.310569,7.0,NaN
1078,3202,997,998,12,2021,Notre Dame,Georgia Tech,87,"Sat, Nov 20",W,False,NaN,285.0,67.0,89.0,0,55,55,True,FBS Indep.,10,6,43,9,11,6.0,NaN,-1.0,1642.428168,6.0,NaN


**3. Data Manipulation - Remove Year 2020**

In [51]:
# Remove Year 2020 from df_base
df_base = df_base[df_base['season'] != '2020']

**3. Data Manipulation - Setup all Binary Variables**

In [52]:
# Note -- may need to remove "Bye" / non-game weeks prior to engaging in this manipulation

# Merge Opponents Conference onto the dataframe:
df_base = df_base.merge(df_conference, how = 'left', left_on = 'opponent', right_on = 'Team ID')
df_base.rename(columns={'Conference ID': 'Opp CONF'}, inplace=True)


# Create binary columns for each conference in the conference list for the home team
for conf in conference_list:
    df_base["team " + conf] = (df_base['CONF'] == conf).astype(int)
    df_base["opp " + conf] = (df_base['Opp CONF'] == conf).astype(int)
    df_base['Same CONF'] = (df_base['CONF'] == df_base['Opp CONF']).astype(int)

# Creating the binary columns
df_base['win_loss'] = (df_base['win_loss'] == 'W').astype(int)
df_base['OT'] = (df_base['OT'] == True).astype(int)
df_base['home_game'] = (df_base['home_game'] == True).astype(int)

df_base.head()

,index,Unnamed: 0,week,season,Team,opponent,code,date,win_loss,OT,OT_num,pass,rush,rec,points_allowed,points_scored,point_differential,home_game,CONF,SOR,FPI,SOS,GC,AVGWP,AP_rank,opponent_rank,rank_change,Elo_pregame,Cus_AP_Rank,elo_rank_unranked,Team ID,Opp CONF,team Mountain West,opp Mountain West,Same CONF,team MAC,opp MAC,team SEC,opp SEC,team Pac-12,opp Pac-12,team FBS Indep.,opp FBS Indep.,team Big 12,opp Big 12,team ACC,opp ACC,team CUSA,opp CUSA,team American,opp American,team Sun Belt,opp Sun Belt,team Big Ten,opp Big Ten
0,2238,2239,2,2021,Air Force,Navy,2005,"Sat, Sep 11",1,0,NaN,49.0,97.0,27.0,3,23,20,0,Mountain West,33,66,110,24,6,NaN,NaN,NaN,1500.000000,26.0,1.0,Navy,American,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,2600,2601,3,2021,Air Force,Utah State,2005,"Sat, Sep 18",0,0,NaN,182.0,102.0,NaN,49,45,-4,1,Mountain West,33,66,110,24,6,NaN,NaN,0.0,1537.802361,28.0,3.0,Utah State,Mountain West,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3007,3008,4,2021,Air Force,Florida Atlantic,2005,"Sat, Sep 25",1,0,NaN,70.0,164.0,46.0,7,31,24,1,Mountain West,33,66,110,24,6,NaN,NaN,0.0,1516.382623,54.0,29.0,Florida Atlantic,American,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,3007,3008,4,2021,Air Force,Florida Atlantic,2005,"Sat, Sep 25",1,0,NaN,70.0,164.0,46.0,7,31,24,1,Mountain West,33,66,110,24,6,NaN,NaN,0.0,1516.382623,54.0,29.0,Florida Atlantic,CUSA,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,3454,3455,5,2021,Air Force,New Mexico,2005,"Sat, Oct 2",1,0,NaN,33.0,142.0,33.0,10,38,28,0,Mountain West,33,66,110,24,6,NaN,NaN,0.0,1547.200041,35.0,10.0,New Mexico,Mountain West,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [53]:
Testing 123

SyntaxError: invalid syntax (2780890730.py, line 1)